In [1]:
%%writefile pool_example.py
import numpy as np
import time
from random import random
from multiprocessing import Pool
from functools import partial


def explicit_matmul(A,B):
    #A[m][n]
    #B[n][p]
    #C[m][p]    
    C_temp = np.zeros((np.shape(A)[0],np.shape(A)[1]))    
    for i in range(np.shape(A)[0]): #(i=1...m) Rows in A
        for j in range(np.shape(B)[1]): # (j=1...p) Columns in B
            for k in range(np.shape(A)[1]): # (k=1...n) Columns in A
                C_temp[i][j] += A[i][k] * B[k][j]
    return(C_temp)
 
def explicit_matmul_thread(A,B,C,k):
    #A[m][n]
    #B[n][p]
    #C[m][p]
    C_temp = np.zeros((np.shape(A)[0],np.shape(A)[1]))
    for i in range(np.shape(A)[0]): #(i=1...m) Rows in A
        for j in range(np.shape(B)[1]): # (j=1...p) Columns in B
            #for k in range(np.shape(A)[1]): # (k=1...n) Columns in A
            C_temp[i][j] += A[i][k] * B[k][j]
    return(C_temp)
 
if __name__ == '__main__':
     
    nprocs= 16
    print(nprocs)
     
    AX=AY=BX=BY=800
    print("Making Matricies")
    A = np.random.rand(AX,AY)
    B = np.random.rand(BX,BY)
    C = np.zeros((AX,AY))
    print("starting procs")
    start_thread = time.perf_counter()
    pool = Pool(nprocs)
    iter_procs = [i for i in range(AX)]
 
    C_parallel = pool.map(partial(explicit_matmul_thread,A,B,C),iter_procs)
    #you need to build C_parallel back again)
    C_parallel = np.sum(C_parallel,axis=0)
 
    pool.close() # Make sure no more processes are submitted to pool
    pool.join() # Wait for the worker processes to finish    
    end_thread = time.perf_counter()
    print("Thread mult: ",end_thread -start_thread)
 
    start_explicit = time.perf_counter()
    C_explicit = explicit_matmul(A,B)
    end_explicit  = time.perf_counter()
    print("Explicit mult: ",end_explicit - start_explicit)
     
    start_np = time.perf_counter()
    C_np = np.matmul(A,B)
    end_np  = time.perf_counter()
    print("NumPy  matmul: ",end_np -start_np)
     
    if not np.allclose(C_explicit, C_parallel, rtol=1e-10, atol=1e-10):
        print("C_parallel is not equal to C_explicit!!")
    if not np.allclose(C_parallel, C_np, rtol=1e-10, atol=1e-10):
        print("C_parallel is not equal to C_np!!")
    if not np.allclose(C_explicit, C_np, rtol=1e-10, atol=1e-10):
        print("C_np is not equal to C_explicit!!")

Overwriting pool_example.py


In [ ]:
!python pool_example.py

16
Making Matricies
starting procs


In [1]:
!hostname

r29n1.lisa.surfsara.nl


In [43]:
99.012/9.27

10.680906148867315

In [14]:
!env | grep "OMP"

EBDEVELGOMPI=/sw/arch/Debian10/EB_production/2021/software/gompi/2021a/easybuild/gompi-2021a-easybuild-devel
OMPI_MCA_orte_tmpdir_base=/dev/shm/
EBROOTGOMPI=/sw/arch/Debian10/EB_production/2021/software/gompi/2021a
EBVERSIONGOMPI=2021a


In [10]:
%%writefile python_job.sh
#!/bin/bash
#SBATCH -n 16
#SBATCH -p normal
#SBATCH -t 00:30:00


module load 2021 SciPy-bundle/2021.05-foss-2021a

python pool_example.py



Overwriting python_job.sh


In [17]:
!pwd
!ls

/home/lcur1225/Untitled Folder
pool_example.py  slurm-8957708.out  slurm-8957930.out  Untitled.ipynb
python_job.sh	 slurm-8957924.out  slurm-8957934.out


In [11]:
!sbatch python_job.sh

Submitted batch job 8957934


In [16]:
!squeue -u lcur1225

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           8957929 gpu_titan hub-jhls lcur1225  R      11:58      1 r29n1
           8957934    normal python_j lcur1225  R       7:23      1 r25n12


In [18]:
!cat slurm-8957934.out

16
Making Matricies
starting procs
Thread mult:  39.26104412204586
Explicit mult:  407.6354851019569
NumPy  matmul:  0.06795122602488846


In [20]:
%%writefile communicator.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

print('Number of processes is %i.' %size)
print('Hello, I am process %i.' % rank)

Writing communicator.py


In [2]:
!mpirun --oversubscribe -np 10 python communicator.py

Number of processes is 10.
Hello, I am process 7.
Number of processes is 10.
Hello, I am process 2.
Number of processes is 10.
Hello, I am process 4.
Number of processes is 10.
Hello, I am process 1.
Number of processes is 10.
Hello, I am process 6.
Number of processes is 10.
Hello, I am process 3.
Number of processes is 10.
Hello, I am process 5.
Number of processes is 10.
Hello, I am process 8.
Number of processes is 10.
Hello, I am process 9.
Number of processes is 10.
Hello, I am process 0.
